In [ ]:
import pandas as pd
import numpy as np 
import googleapiclient

import json, jsonlines, time
from googleapiclient.http import HttpError
from tqdm import tqdm

In [ ]:
ATTRIBUTES = ["TOXICITY", "SEVERE_TOXICITY", "IDENTITY_ATTACK", "INSULT", "PROFANITY", "THREAT"]
BASE_DIR = "/extra/ucinlp1/cbelem/experiment-ro-prompts/generations-results/uncond-half-multinomial/EleutherAI__pythia-1.4b"
OUTPUT_DIR = f"{BASE_DIR}/raw_data__perspective_AI.jsonl"
ORIG_PREFIX_PATH = f"{BASE_DIR}/raw_data.csv"


def setup_perspective_api_client(path: str) -> str:
    from googleapiclient import discovery

    with open(path, "r") as f:
        api_key = f.read()
    
    client = discovery.build(
        "commentanalyzer",
        "v1alpha1",
        developerKey=api_key,
        discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
        static_discovery=False,
    )
    return client


client = setup_perspective_api_client("/home/cbelem/projects/constrained-decoding/experiment-ro-prompts/perspective_api.txt")
client

In [ ]:
import traceback as tb

def get_toxicity(seq, client, attributes=ATTRIBUTES):
    def try_request(i=0) -> bool:
        analyze_request = {
            'comment': { 'text': seq },
            'requestedAttributes': {attr: {} for attr in attributes},
            "spanAnnotations": False,
            "doNotStore": True,
        }
        try:
             return client.comments().analyze(body=analyze_request).execute(), i
        except HttpError:
            tb.print_exc()
            return None, i+5
        
    
    result, time_incr = None
    while result is None:
        result, time_incr = try_request(time_incr)
        if result is None:
            print("Trying again in...", 60+time_incr)
            time.sleep(60+time_incr)

    return result

In [ ]:
def collect_toxicity(data: pd.DataFrame, prefix_col: str="prefix", sequence_col: str ="sequence"):
    results = []
    for pref, seq in tqdm(zip(data[prefix_col].values, data[sequence_col].values)):
        r = get_toxicity(pref, client)
        r["prefix"] = pref
        r["sequence"] = seq   
        results.append(r)
    return results

In [ ]:
d = pd.read_csv(ORIG_PREFIX_PATH, index_col=0)
toxic = collect_toxicity(d, "prefix", "sequence")

with jsonlines.open(OUTPUT_DIR, mode='w') as writer:
    writer.write_all(results)


In [ ]:
d["prefix_toxicity"]

In [ ]:
r["prefix"] = seqs[0]

In [ ]:
!ls /extra/ucinlp1/cbelem/experiment-ro-prompts/generations-results/cond-roprompt/

In [ ]:
r

In [ ]:
d